In [2]:
import numpy as np
from roboticstoolbox import ETS, ERobot, ET
from spatialmath import SE3
from scipy.linalg import inv

# Define your robot's Denavit-Hartenberg parameters or use ETS
# For this example, I'll create a generic 6-DOF robot arm based on your previous code

# Transformation dictionary for joint axes
transform_dict = {
    'z': ET.Rz,
    'x': ET.Rx,
    'y': ET.Ry
}

# Define the axes of the joints
axes = ['z', 'y', 'z', 'y', 'z', 'y']  # Example configuration

# Define link lengths (you can adjust these as needed)
link_lengths = [0.2] * 6  # Example link lengths

# Build the robot model
def build_robot(link_lengths):
    ets = ETS()
    for length, axis in zip(link_lengths, axes):
        ets *= transform_dict[axis]()  # Revolute joint
        ets *= ET.tz(length)  # Translation along z-axis (link length)
    robot = ERobot(ets, name='6-Link Robot Arm')

    # Define link inertias and masses (assuming cylindrical links)
    for i in range(6):
        robot.links[i].m = np.pi * (0.03 ** 2) * link_lengths[i]  # Mass (density * volume)
        robot.links[i].r = [0, 0, link_lengths[i] / 2]  # Center of mass at half the link
        # Inertia tensor for a cylinder along z-axis
        Ixx = (1/12) * robot.links[i].m * (3 * (0.03 ** 2) + link_lengths[i] ** 2)
        Iyy = Ixx
        Izz = (1/2) * robot.links[i].m * (0.03 ** 2)
        robot.links[i].I = np.diag([Ixx, Iyy, Izz])  # Inertia matrix
    return robot

# Desired end-effector pose (example)
T = SE3(0.5, 0.2, 0.3) * SE3.RPY([0.1, 0.2, 0.3])

# Build the robot
robot = build_robot(link_lengths)

min = 1e6
final_q = None
for i in range(1):
    # Solve inverse kinematics to find joint angles q
    ik_result = robot.ikine_LM(T, mask=[1, 1, 1, 1, 1, 0])  # Mask to ignore the last joint
    if not ik_result.success:
        print("Inverse kinematics failed to find a solution.")
    else:
        q = ik_result.q  # Joint angles
        OSIM = robot.inertia_x(q)  # Operational space inertia matrix
        OSIM_eig = np.linalg.eig(OSIM)[0]  # Eigenvalues of OSIM
        min = np.min([min, np.min(OSIM_eig)])
        final_q = q
min

1.6209406581812732e-06

In [56]:
final_q

array([-2.17276358, -1.06301772,  2.44396571,  1.69257553,  1.09278467,
       -2.18387829])

In [ ]:
Nfeval = 1

def callbackF(Xi, *conv):
    global Nfeval
    print(Nfeval, Xi, conv)
    Nfeval += 1

axes = ['z', 'y', 'z', 'y', 'z', 'y']  # Standard configuration

transform_dict = {
    'z': ET.Rz,
    'x': ET.Rx,
    'y': ET.Ry
}

# define inertia tensor for cylinder
def inertia_tensor_cylinder(radius, height):
    # Inertia tensor for a solid cylinder about its central axis
    mass = np.pi * radius**2 * height
    I_xx = (1/12) * mass * (3 * radius**2 + height**2)
    I_yy = (1/12) * mass * (3 * radius**2 + height**2)
    I_zz = (0.5) * mass * radius**2
    return np.array([[I_xx, 0, 0],
                     [0, I_yy, 0],
                     [0, 0, I_zz]])

mask = np.array([1, 1, 1, 1, 1, 0])
T = SE3(0.5, 0.0, 0.1) * SE3.Rx(-np.pi)

def manipulability_objective(link_lengths):
    # print(link_lengths)
    # Build the robot model with the given link lengths
    ets = ETS()
    for length, axis in zip(link_lengths, axes):
        ets *= transform_dict[axis]()
        ets *= ET.tz(length)
    robot = ERobot(ets, name='6-Link Robot Arm')    
    
    for i in range(6):
        robot.links[i].r = [0, 0, link_lengths[i]/2]
        robot.links[i].I = inertia_tensor_cylinder(0.03, link_lengths[i])
        robot.links[i].m = np.pi * 0.03 ** 2 * link_lengths[i]

    # Solve IK
    ik_result = robot.ikine_LM(T, mask=mask)
    if not ik_result.success:
        # print(link_lengths)
        # print("Inverse kinematics failed to find a solution.")
        return 1e6
    q = ik_result.q
    
    OSIM = robot.inertia_x(q)
    OSIM_trace = np.trace(OSIM)
    
    return OSIM_trace

l_tot_max = 2
l_tot_min = 1

initial_link_lengths = np.array([0.2] * 6)
bounds = [(0.1, 0.2)] + [(0.1, 1)] * 5
print(bounds)
from scipy.optimize import differential_evolution, basinhopping, minimize, shgo, dual_annealing, least_squares

result = differential_evolution(manipulability_objective, bounds, workers=-1, maxiter=1, popsize=10, recombination=0.9, mutation=0.2)
# result = basinhopping(manipulability_objective, x0=initial_link_lengths, niter=100, disp=True, callback=callbackF)
# result = shgo(manipulability_objective, bounds, n=100, iters=5, sampling_method='sobol', disp=True)
# result = minimize(manipulability_objective, x0=initial_link_lengths, method='Nelder-Mead', options={'disp': True})
# result = dual_annealing(manipulability_objective, bounds, maxiter=100, callback=callbackF, x0=initial_link_lengths)
result

[(0.1, 0.2), (0.1, 1), (0.1, 1), (0.1, 1), (0.1, 1), (0.1, 1)]


C:\Users\twang\PycharmProjects\BEP_notebooks\.venv\Lib\site-packages\scipy\optimize\_differentialevolution.py:488: UserWarning: differential_evolution: the 'workers' keyword has overridden updating='immediate' to updating='deferred'
  with DifferentialEvolutionSolver(func, bounds, args=args,
